In [110]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import patsy
import seaborn as sns

from sklearn.cross_validation import train_test_split
import seaborn as sns

exp_values = [
    'Pclass',
    #     'PassengerId',
    'C(Sex)',
    'C(Child)',
    'C(Embarked)',
    #     'SibSp',
]
predict_value = 'Survived'
child_threthold = 16


def output_result(model, category_data, base_data):
    result = model.predict(category_data)
    with open('data/result.csv', 'w') as f:
        f.write('PassengerId,Survived\n')
        for r, pid in zip(result, base_data['PassengerId']):
            f.write('{},{}\n'.format(pid, str(int(r))))


def category_to_table(titanic, is_training_data=True):
    fomula = '+'.join(exp_values)
    if is_training_data:
        fomula = fomula + '+' + predict_value

    print('fomula:', fomula)

    c_df = patsy.dmatrix(fomula, data=titanic, return_type='dataframe')
    return c_df


def load_titanic_data(is_training_data=True):
    """
    データを読みこみ、
    必要なパラメータに絞る
    """

    s_rate_ticket_dict = dict()
    s_count_ticket_dict = dict()

    def is_child(age):
        if pd.isnull(age):
            return False
        if age < child_threthold:
            return True
        return False

    def ticket_count_info(x):
        if x in s_count_ticket_dict:
            return s_count_ticket_dict[x]
        return 0

    def ticket_ratio_info(x):
        if x in s_rate_ticket_dict:
            return s_rate_ticket_dict[x]
        return None

    titanic = None
    if is_training_data:
        titanic = pd.read_csv("data/train.csv")
    else:
        titanic = pd.read_csv("data/test.csv")

    if is_training_data:
        subset_ = ['Age', 'Pclass']
        titanic = titanic.dropna(
            subset=subset_
        )

        for k, v in titanic.groupby('Ticket')[['Survived']].count().items():
            print(k)
            for kk, vv in v.items():
                s_count_ticket_dict[kk] = vv

        for k, v in titanic.groupby('Ticket')[['Survived']].mean().items():
            print(k)
            for kk, vv in v.items():
                s_rate_ticket_dict[kk] = vv

        titanic = titanic.assign(
            TicketCount=titanic['Ticket'].map(ticket_count_info),
            TicketSRate=titanic['Ticket'].map(ticket_ratio_info),
            Type="Train",
            Training=1,
            Test=0
        )
    else:
        titanic = titanic.assign(
            Surrvived=None,
            Type="Test",
            Training=0,
            Test=1
        )

    titanic = titanic.assign(
        Child=titanic['Age'].map(is_child),
    )
    return titanic


def hit_rate(predict, test):
    N = len(predict)
    hit = sum([1 if p == t else 0 for p, t in zip(predict, test)])
    return "{0:.3f}%".format(100 * hit / N * 1.0)


def main():
    titanic = load_titanic_data()
    titanic_category = category_to_table(titanic)

    x_columns = list(titanic_category.columns)
    x_columns.remove(predict_value)

    X_train, X_test, Y_train, Y_test = train_test_split(titanic[x_columns],
                                                        titanic[predict_value],
                                                        test_size=0.4,
                                                        random_state=0)


titanic = load_titanic_data()
titanic.head(2)

## カテゴリカルなデータに変換
titanic_category = category_to_table(titanic)
titanic_category.head(2)

test_data = load_titanic_data(False)
test_data_category = category_to_table(test_data, False)

full_data = pd.concat([titanic, test_data])

Survived
Survived
fomula: Pclass+C(Sex)+C(Child)+C(Embarked)+Survived
fomula: Pclass+C(Sex)+C(Child)+C(Embarked)


In [104]:
s_rate_ticket_dict = dict()
s_count_ticket_dict = dict()
for k, v in titanic.groupby('Ticket')[['Survived']].count().items():
    print(k)
    for kk, vv in v.items():
#         if vv > 3:
        s_count_ticket_dict[kk] = vv

for k, v in titanic.groupby('Ticket')[['Survived']].mean().items():
    print(k)
    for kk, vv in v.items():
        s_rate_ticket_dict[kk] = vv

def ticket_count_info(x):
    key = x["Ticket"]
    if key in s_count_ticket_dict:
        return s_count_ticket_dict[key]

def ticket_ratio_info(x):
    key = x["Ticket"]
    if key in s_rate_ticket_dict:
        return s_rate_ticket_dict[key]


Survived
Survived


In [105]:
x_columns = list(titanic_category.columns)
x_columns.remove(predict_value)
x_columns

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit


X_train, X_test, Y_train, Y_test = train_test_split(titanic_category[x_columns],
                                                                                        titanic_category[predict_value],
                                                                                        test_size=0.4,
                                                                                        random_state=0)

print(len(X_train), len(X_test))

train_user = set(X_train.index)

training_titanic = titanic.query('PassengerId in @train_user')
test_titanic = titanic.query('PassengerId not in @train_user')


427 285


In [128]:
ts_1_table = titanic[(titanic['TicketCount'] > 2) & (titanic["TicketSRate"] >= 0.75)][["Ticket", "Survived", "TicketCount", "TicketSRate", "TicketSRate"]]
print(ts_1_table[['Survived']].mean())

ts_0_table = titanic[(titanic['TicketCount'] > 2) & (titanic["TicketSRate"] <= 0.25)][["Ticket","Survived", "TicketCount", "TicketSRate", "TicketSRate"]]
print(ts_0_table[['Survived']].mean())


Survived    0.95
dtype: float64
Survived    0.02
dtype: float64


In [136]:
survived_ticket_set = set()
dead_ticket_set = set()

for ticket in ts_1_table.Ticket.values:
    survived_ticket_set.add(ticket)

for ticket in ts_0_table.Ticket.values:
    dead_ticket_set.add(ticket)

survived_ticket_set
dead_ticket_set


{'3101295',
 '345773',
 '347082',
 '347088',
 '349909',
 '382652',
 'CA 2144',
 'LINE',
 'S.O.C. 14879',
 'W./C. 6608'}